In [2]:
import numpy as np
from shapely.geometry import Polygon, Point
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
import math

def asignar_repas_envios(vertices_poligono, cantidad_repas, cantidad_envios, n_clusters, random_state=None):
    """
    Asigna repartidores a envíos utilizando clustering y minimización de distancias geográficas.
    Utiliza vectorización con NumPy para mejorar los tiempos de cálculo de la matriz de distancias.
    
    Parámetros:
    - vertices_poligono: Lista de tuplas (longitud, latitud) que representan los vértices del polígono.
    - cantidad_repas: Número de repartidores a generar.
    - cantidad_envios: Número de envíos a generar.
    - n_clusters: Número inicial de clusters para KMeans.
    - random_state: Semilla para la generación de números aleatorios (int, None o np.random.RandomState).
    
    Retorna:
    - asignaciones: Lista de tuplas (repa_id, envio_id).
    - sobrantes_repas: Lista de IDs de repartidores no asignados.
    - sobrantes_envios: Lista de IDs de envíos no asignados.
    - total_asignaciones: Número total de asignaciones realizadas.
    - distancia_total_metros: Suma total de las distancias de todas las asignaciones en metros.
    """
    # Crear el polígono
    poligono = Polygon(vertices_poligono)

    # Crear un generador de números aleatorios
    rng = np.random.default_rng(random_state)

    # Función para generar puntos aleatorios dentro del polígono
    def generar_puntos_en_poligono(poligono, cantidad):
        minx, miny, maxx, maxy = poligono.bounds
        puntos = []
        while len(puntos) < cantidad:
            random_point = Point(rng.uniform(minx, maxx), rng.uniform(miny, maxy))
            if poligono.contains(random_point):
                puntos.append(random_point)
        return puntos

    # Función vectorizada para calcular la matriz de distancias Haversine
    def matriz_distancias_haversine(repas_cluster, envios_cluster):
        # Convertir listas de puntos a arrays de coordenadas
        lat1 = np.radians(np.array([p.y for p in repas_cluster]))[:, np.newaxis]
        lon1 = np.radians(np.array([p.x for p in repas_cluster]))[:, np.newaxis]
        lat2 = np.radians(np.array([p.y for p in envios_cluster]))[np.newaxis, :]
        lon2 = np.radians(np.array([p.x for p in envios_cluster]))[np.newaxis, :]

        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

        R = 6371000  # Radio de la Tierra en metros
        distancia = R * c
        return distancia  # Retorna la matriz de distancias en metros

    # Generar ubicaciones aleatorias para repas y envíos
    puntos_repas = generar_puntos_en_poligono(poligono, cantidad_repas)
    puntos_envios = generar_puntos_en_poligono(poligono, cantidad_envios)

    # Puntos mios para validar
    # puntos_repas = [Point(-34.57875, -58.4574), Point(-34.61499, -58.47551), Point(-34.59987, -58.41337), Point(-34.5804, -58.42911)]
    # puntos_envios = [Point(-34.58567, -58.46624), Point(-34.62064, -58.46504), Point(-34.61047, -58.40599), Point(-34.59961, -58.44388)]
    

    # Asignar IDs
    ids_repas = list(range(len(puntos_repas)))
    ids_envios = list(range(len(puntos_envios)))

    # Combinar todos los puntos para clustering
    puntos_todos = puntos_repas + puntos_envios
    etiquetas = ['repa'] * len(puntos_repas) + ['envio'] * len(puntos_envios)
    ids_todos = ids_repas + ids_envios  # IDs correspondientes a puntos_todos

    # Obtener coordenadas para clustering
    coordenadas = np.array([[p.x, p.y] for p in puntos_todos])

    # Realizar clustering inicial
    print(f"\n--- Primera Agrupación con {n_clusters} clusters ---")
    n_clusters_current = min(n_clusters, len(coordenadas))
    kmeans = KMeans(n_clusters=n_clusters_current, random_state=random_state)
    clusters = kmeans.fit_predict(coordenadas)

    # Inicializar listas de asignaciones y sobrantes
    asignaciones = []
    sobrantes_repas = ids_repas.copy()
    sobrantes_envios = ids_envios.copy()

    distancia_total_metros = 0  # Para acumular la distancia total de las asignaciones en metros

    # Procesar cada cluster
    for cluster_id in range(n_clusters_current):
        # Obtener índices de puntos en el cluster actual
        indices_cluster = np.where(clusters == cluster_id)[0]

        # Separar repas y envíos en el cluster
        repas_cluster = []
        envios_cluster = []
        ids_repas_cluster = []
        ids_envios_cluster = []

        for idx in indices_cluster:
            etiqueta = etiquetas[idx]
            if etiqueta == 'repa':
                repas_cluster.append(puntos_todos[idx])
                ids_repas_cluster.append(ids_todos[idx])
            else:
                envios_cluster.append(puntos_todos[idx])
                ids_envios_cluster.append(ids_todos[idx])

        # Mostrar información del cluster
        print(f"\nCluster {cluster_id}:")
        print(f" - Repartidores en el cluster: {len(repas_cluster)}")
        print(f" - Envíos en el cluster: {len(envios_cluster)}")

        # Si no hay repas o envíos en el cluster, continuar
        if len(repas_cluster) == 0 or len(envios_cluster) == 0:
            print(" - Cluster sin asignaciones posibles.")
            continue

        # Crear matriz de distancias utilizando vectorización
        matriz_distancias = matriz_distancias_haversine(repas_cluster, envios_cluster)

        # Resolver el problema de asignación utilizando el algoritmo húngaro
        filas_ind, columnas_ind = linear_sum_assignment(matriz_distancias)

        num_asignaciones_cluster = min(len(filas_ind), len(columnas_ind))
        print(f" - Asignaciones en este cluster: {num_asignaciones_cluster}")

        # Acumular distancia total y realizar asignaciones
        for idx in range(num_asignaciones_cluster):
            i = filas_ind[idx]
            j = columnas_ind[idx]
            repa_id = ids_repas_cluster[i]
            envio_id = ids_envios_cluster[j]
            asignaciones.append((repa_id, envio_id))
            distancia_total_metros += matriz_distancias[i, j]
            print('caca: ', matriz_distancias[i, j])
            # Remover asignados de los sobrantes
            if repa_id in sobrantes_repas:
                sobrantes_repas.remove(repa_id)
            if envio_id in sobrantes_envios:
                sobrantes_envios.remove(envio_id)

    # Mostrar estado después de la primera agrupación
    print(f"\n--- Estado después de la primera agrupación ---")
    print(f"Total de asignaciones: {len(asignaciones)}")
    print(f"Repartidores sobrantes: {len(sobrantes_repas)}")
    print(f"Envíos sobrantes: {len(sobrantes_envios)}")

    # Si hay sobrantes, volver a clusterizar
    iteracion = 1
    asignaciones_en_iteracion = len(asignaciones)
    while sobrantes_repas and sobrantes_envios and n_clusters > 1:
        iteracion += 1

        # Si no hubo asignaciones en la iteración anterior, reducir el número de clusters
        if asignaciones_en_iteracion == 0:
            n_clusters -= 1
            print(f"\nNo hubo asignaciones en la iteración anterior. Reduciendo clusters a {n_clusters}")

        asignaciones_en_iteracion = 0  # Reiniciar el contador de asignaciones en esta iteración

        print(f"\n--- Reagrupación de sobrantes: Iteración {iteracion} ---")

        # Actualizar puntos y etiquetas de sobrantes
        puntos_repas_sobrantes = [puntos_repas[ids_repas.index(i)] for i in sobrantes_repas]
        puntos_envios_sobrantes = [puntos_envios[ids_envios.index(i)] for i in sobrantes_envios]

        ids_repas_sobrantes = sobrantes_repas.copy()
        ids_envios_sobrantes = sobrantes_envios.copy()

        puntos_todos_sobrantes = puntos_repas_sobrantes + puntos_envios_sobrantes
        etiquetas_sobrantes = ['repa'] * len(puntos_repas_sobrantes) + ['envio'] * len(puntos_envios_sobrantes)
        ids_todos_sobrantes = ids_repas_sobrantes + ids_envios_sobrantes

        # Actualizar coordenadas
        coordenadas_sobrantes = np.array([[p.x, p.y] for p in puntos_todos_sobrantes])

        # Calcular el número actual de clusters
        n_clusters_current = min(n_clusters, len(coordenadas_sobrantes))
        print(f" - Número de clusters en esta iteración: {n_clusters_current}")

        # Evitar clusters vacíos
        if n_clusters_current <= 0:
            break

        # Re-clusterizar sobrantes
        kmeans = KMeans(n_clusters=n_clusters_current, random_state=random_state)
        clusters_sobrantes = kmeans.fit_predict(coordenadas_sobrantes)

        # Procesar cada cluster de sobrantes
        for cluster_id in range(n_clusters_current):
            indices_cluster = np.where(clusters_sobrantes == cluster_id)[0]

            repas_cluster = []
            envios_cluster = []
            ids_repas_cluster = []
            ids_envios_cluster = []

            for idx in indices_cluster:
                etiqueta = etiquetas_sobrantes[idx]
                if etiqueta == 'repa':
                    repas_cluster.append(puntos_todos_sobrantes[idx])
                    ids_repas_cluster.append(ids_todos_sobrantes[idx])
                else:
                    envios_cluster.append(puntos_todos_sobrantes[idx])
                    ids_envios_cluster.append(ids_todos_sobrantes[idx])

            # Mostrar información del cluster
            print(f"\nCluster {cluster_id}:")
            print(f" - Repartidores en el cluster: {len(repas_cluster)}")
            print(f" - Envíos en el cluster: {len(envios_cluster)}")

            if len(repas_cluster) == 0 or len(envios_cluster) == 0:
                print(" - Cluster sin asignaciones posibles.")
                continue

            # Crear matriz de distancias utilizando vectorización
            matriz_distancias = matriz_distancias_haversine(repas_cluster, envios_cluster)

            filas_ind, columnas_ind = linear_sum_assignment(matriz_distancias)

            num_asignaciones_cluster = min(len(filas_ind), len(columnas_ind))
            print(f" - Asignaciones en este cluster: {num_asignaciones_cluster}")

            # Acumular distancia total y realizar asignaciones
            for idx in range(num_asignaciones_cluster):
                i = filas_ind[idx]
                j = columnas_ind[idx]
                repa_id = ids_repas_cluster[i]
                envio_id = ids_envios_cluster[j]
                asignaciones.append((repa_id, envio_id))
                distancia_total_metros += matriz_distancias[i, j]
                print('caca: ', matriz_distancias[i, j])
                asignaciones_en_iteracion += 1
                if repa_id in sobrantes_repas:
                    sobrantes_repas.remove(repa_id)
                if envio_id in sobrantes_envios:
                    sobrantes_envios.remove(envio_id)

        # Mostrar estado después de la iteración
        print(f"\nEstado después de la iteración {iteracion}:")
        print(f"Total de asignaciones: {len(asignaciones)}")
        print(f"Repartidores sobrantes: {len(sobrantes_repas)}")
        print(f"Envíos sobrantes: {len(sobrantes_envios)}")

    total_asignaciones = len(asignaciones)
    return asignaciones, sobrantes_repas, sobrantes_envios, total_asignaciones, distancia_total_metros

In [3]:
vertices= ((-34.6251522,-58.4132406),
(-34.6208589,-58.379557),
(-34.6149255,-58.3792137),
(-34.5968403,-58.3994698),
(-34.5745109,-58.3960365),
(-34.5452476,-58.4434151),
(-34.5682916,-58.4669327),
(-34.5686057,-58.49595),
(-34.5946108,-58.505563),
(-34.6233159,-58.4879133),
(-34.6398418,-58.4599325),
(-34.6320736,-58.4448263),
(-34.6251522,-58.4132406))

In [6]:
asignaciones, sobrantes_repas, sobrantes_envios, total_asignaciones, distancia_total_metros = asignar_repas_envios(vertices, 2000, 2000, 1, 123)



--- Primera Agrupación con 1 clusters ---

Cluster 0:
 - Repartidores en el cluster: 2000
 - Envíos en el cluster: 2000
 - Asignaciones en este cluster: 2000
caca:  248.06949383406112
caca:  75.09435104079134
caca:  315.9458820030739
caca:  44.35837485746498
caca:  62.29816567072522
caca:  159.83739938630524
caca:  180.51521001279843
caca:  151.65676657003988
caca:  243.85599674008515
caca:  166.4767078713302
caca:  103.24905795274474
caca:  202.4918283697701
caca:  69.34716257574627
caca:  74.38963013439711
caca:  128.7459788292156
caca:  223.17195458279286
caca:  281.27890620505644
caca:  63.03662387870355
caca:  148.11841662277632
caca:  631.6048032812075
caca:  139.08450268794965
caca:  181.59560303552757
caca:  67.54898775362844
caca:  94.8387592190013
caca:  135.07861659547078
caca:  96.85944434501926
caca:  193.6400896723859
caca:  168.54304172749096
caca:  277.2617637851333
caca:  194.13135302139665
caca:  31.393224517012833
caca:  166.67625370761593
caca:  51.29861648157711
c